In [5]:
# coding: utf-8
from typing import Dict, List

import pandas as pd


class Datos:
    datos: int
    nominalAtributos: List[bool]

    # Constructor: procesar el fichero para asignar correctamente las variables nominalAtributos, datos y diccionarios
    def __init__(self, nombreFichero: str):
        self.datosCrudos = pd.read_csv(nombreFichero)
        self.datos = self.datosCrudos.copy()

        self.nominalAtributos = []
        self.diccionarios = {}

        for columna in self.datos.columns:
            if self._es_nominal(columna):
                self.nominalAtributos.append(True)
                self.diccionarios[columna] = self._generar_mapeo(columna)
                self.datos[columna] = self._reemplazar_valores(columna)
            elif self._es_numerico(columna):
                self.nominalAtributos.append(False)
                self.diccionarios[columna] = {}
            else:
                raise ValueError(
                    f"La columna '{columna}' contiene valores que no son nominales ni enteros/decimales."
                )

    def _es_nominal(self, columna: str) -> bool:
        # es verdadero si es la columna objetivo o si sus valores son nominales
        return columna.lower() == "class" or self.datos[columna].dtype.name == "object"

    def _es_numerico(self, columna: str) -> bool:
        # es verdadero si los valores son un n�meros enteros o reales
        return self.datos[columna].dtype.name in ["int64", "float64"]

    def _generar_mapeo(self, columna_nominal: str):
        # se extraen los valores �nicos de la columna y se sortean lexicograficamente
        valores = [str(valor) for valor in self.datos[columna_nominal].unique()]
        valores = sorted(valores)

        return {valor: indice for indice, valor in enumerate(valores)}

    def _reemplazar_valores(self, columna_nominal: str) -> pd.Series:
        mapeo = self.diccionarios[columna_nominal]
        return self.datos[columna_nominal].map(lambda valor: mapeo[str(valor)])

    # Devuelve el subconjunto de los datos cuyos �ndices se pasan como argumento
    def extraeDatos(self, idx: List[int]):
        return self.datos.iloc[idx]


In [9]:
import math
import random
import numpy as np
datos = Datos("wdbc.csv").datos


# normalizar los datos
for col in datos.columns:
    datos[col] = datos[col] / datos[col].abs().max()


# crea umbral
umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
print("El umbral: ", umbral)

# constante de aprendizaje
k_apre = 1

# entra w.x
def sigma(w_v, x_v):
    x = np.array(x_v)[0]
    w = np.array(w_v)[0]
    #print(w, x)
    return 1 / 1 + math.exp(-w*x)


def actualiza_w(w, x):
    t = x.iloc[0,-1]
    print(t)
    x = np.array(x.iloc[:,:-1])[0]
    #print(w)
    return w - x * (sigma(w, x) - t) * k_apre

print("w actualizada: ", actualiza_w(umbral, datos.head(1)))

#print(np.array(datos.head(1))[0])

class ClasificadorRegresionLogistica:

    def __init__(self, datos, k_apre):
        self.datos = datos
        self.k_apre = k_apre
        self.umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
    
    # normalizar los datos
    for col in datos.columns:
        datos[col] = datos[col] / datos[col].abs().max()

    umbral = [random.uniform(-0.5, 0.5) for _ in datos.columns[:-1]]
    k_apre = 1 # constante de aprendizaje
    
    def sigma(self, w_v, x_v):
        x = np.array(x_v)[0]
        w = np.array(w_v)[0]
        return 1 / 1 + math.exp(-w*x)


    def actualiza_w(self, w, x):
        t = x.iloc[0,-1]
        print(t)
        x = np.array(x.iloc[:,:-1])[0]
        return w - x * (sigma(w, x) - t) * k_apre
    


test = ClasificadorRegresionLogistica(datos.head(1), 1).actualiza_w(umbral, datos)
print(test)

El umbral:  [0.34260315817468356, -0.06307986189199666, -0.3317637609214942, -0.37888166291851855, -0.3731288944319263, 0.2835066730788499, 0.3295387070584259, -0.2042243960517126, 0.21571443105450672, 0.49983337217359824, 0.4400160046640512, 0.2140847384511162, -0.08737937259636397, -0.2957922374031742, -0.13268073697612848, 0.4664380955148898, 0.03080353724548135, 0.3647341385983526, 0.24505851526525502, -0.0048638269312047955, -0.14852822157441525, -0.35518892625753595, -0.1058277721676587, 0.12859644135031223, -0.49869128602091584, 0.3866947538214286, -0.020750257001277195, -0.422208987662524, 0.06961682569360172, -0.31618076169723486]
1.0
w actualizada:  [-0.17137707 -0.27530751 -0.85495818 -0.70031912 -0.95506559 -0.36195911
 -0.23516115 -0.79139025 -0.42334078 -0.14890372  0.13392217  0.06525009
 -0.40120688 -0.52300982 -0.2977663   0.17556209 -0.07816415  0.12329851
 -0.06041903 -0.17154186 -0.71409375 -0.63613221 -0.69601276 -0.25257024
 -1.08388787 -0.11855224 -0.47740793 -1.